# TP1


## Propósito do trabalho


O propósito deste trabalho é a análise de problemas de alocação usando técnicas de SAT,  em lógica proposicional, e IP em lógica linear inteira.


## Enunciado

### Problema 1


1. Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:

    1. Cada reunião ocupa uma sala (enumeradas $1...S\,$) durante um “slot”  $1..T\,$ $(\text{hora},\text{dia})$.  
    2.  Cada reunião tem associado um projeto (enumerados $1..P$) e um conjunto de participantes. Os diferentes colaboradores são enumerados $1..C$.
    3. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. 
    4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  $50\%$ do total de colaboradores do projeto.
    
   São “inputs” do problema:
      1. Os parâmetros $S\,,\,T\,,\,P\,,\,C$
      2. O conjunto de colaboradores de cada projeto, o seu líder e o número de reuniões semanais.
      3. A disponibilidade de cada participante, incluindo o lider. Essa disponibilidade é um conjunto de “slots"  representada numa matriz booleana de acessibilidade com uma linha por cada participante $1..C$ e uma coluna por “slot” $\,1..T\,$
   
   São critérios de optimização:
      1. Maximizar o número de reuniões efetivamente realizadas
      2. Minimizar o número médio de reuniões por participante.


## Para a resolução deste problema encontramos as seguintes restrições
### Restrição 1 - A assiduidade a cada reunião tem de ser superior a 50%
Seja $ x_{c,t} $ a variável que verifica se um colaborador está disponível no slot $ t $ tem-se que:

$\forall t \in T, \frac{(\sum_{i=1, i<c} x_{c,t})} {c} >= 0.5 $



In [1]:
from ortools.linear_solver import pywraplp
from random import randint

horario = pywraplp.Solver.CreateSolver('SCIP')

S = 2
P = 2
C = 4
#{colaborador: {proj: 0 ou 1}}
CP = {0: {0: 1, 1: 1}, 1: {0: 1, 1: 0}, 2: {0: 0, 1: 0}, 3: {0: 1, 1: 1}}
#{lider: {proj: 0 ou 1}}
LP = {0:1, 1:0}
reunioes_semanais = 10


def gera_slots(d,h_min, h_max):
    slots = []
    for i in range(d):
        for j in range(h_min, h_max):
            slots.append((i,j))
    return slots

T = gera_slots(5,9,18)
    
def gera_disponibilidade(C,T):
    disp = {}
    for c in range(C):
        disp[c] = {}
        for t in T:
            disp[c][t] = randint(0,1)
    return disp

disp = gera_disponibilidade(C,T)
    





{0: {(0, 9): 1, (0, 10): 0, (0, 11): 1, (0, 12): 1, (0, 13): 1, (0, 14): 0, (0, 15): 1, (0, 16): 1, (0, 17): 1, (1, 9): 1, (1, 10): 0, (1, 11): 0, (1, 12): 1, (1, 13): 0, (1, 14): 0, (1, 15): 0, (1, 16): 0, (1, 17): 0, (2, 9): 1, (2, 10): 1, (2, 11): 0, (2, 12): 0, (2, 13): 0, (2, 14): 1, (2, 15): 0, (2, 16): 0, (2, 17): 1, (3, 9): 0, (3, 10): 0, (3, 11): 0, (3, 12): 0, (3, 13): 0, (3, 14): 1, (3, 15): 0, (3, 16): 1, (3, 17): 1, (4, 9): 1, (4, 10): 0, (4, 11): 0, (4, 12): 0, (4, 13): 0, (4, 14): 0, (4, 15): 0, (4, 16): 1, (4, 17): 0}, 1: {(0, 9): 1, (0, 10): 0, (0, 11): 1, (0, 12): 0, (0, 13): 0, (0, 14): 1, (0, 15): 0, (0, 16): 1, (0, 17): 1, (1, 9): 0, (1, 10): 1, (1, 11): 1, (1, 12): 0, (1, 13): 1, (1, 14): 0, (1, 15): 1, (1, 16): 1, (1, 17): 1, (2, 9): 1, (2, 10): 0, (2, 11): 0, (2, 12): 0, (2, 13): 0, (2, 14): 1, (2, 15): 1, (2, 16): 0, (2, 17): 0, (3, 9): 1, (3, 10): 1, (3, 11): 1, (3, 12): 0, (3, 13): 0, (3, 14): 0, (3, 15): 0, (3, 16): 0, (3, 17): 0, (4, 9): 1, (4, 10): 1, (4, 

In [9]:
#Criação das matrizes                       
#Matriz de disponibilidade para as reuniões
R = {}
for c in range(C):
    R[c] = {}
    for p in range(P):
        R[c][p] = {}
        for s in range(S):
            R[c][p][s] = {}
            for t in T:
                R[c][p][s][t] = horario.BoolVar(f'R[{c},{p},{s},{t}]')

#Matriz de alocação das reuniões
A = {}
for p in range(P):
    A[p] = {}
    for s in range(S):
        A[p][s] = {}
            for t in T:
                A[p][s][t] = horario.BoolVar(f'A[{p},{s},{t}]')
                

In [ ]:
#Condições

#A assiduidade a cada reunião tem de ser superior a 50%
for p in range(P):
    for s in range(S):
        for t in T:
            for c in range(C):
                solver.Add ( (sum(R[c][p][s][t]) / C) >= 0.5 )
                
#O Líder tem que ir a todas as reuniões do projeto no qual é líder 
for c in LP:
    for p in LP[c]:
        for s in range(S):
            solver.Add( (sum(R[c][p][s][t] for t in range(T)) * A[p][s][t]) == reunioes_semanais)
            
#Cada projeto tem x reuniões semanais
for p in range(P):
    for s in range(S):
        for t in T:
            solver.Add( sum(A[p][s][t]) == reunioes_semanais)
    
#Reuniões em slots iguais, têm que ser feitas em salas diferentes
for t in T:
    for p in range(P):
        solver.Add( sum(A[p][s][t] for s in range(S)) == )
        
